In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

from keras.applications import VGG16
import keras
from keras import backend as K
# vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet = keras.applications.resnet50.ResNet50(include_top=False, 
                                              weights='imagenet', input_shape=(224, 224, 3))
K.set_learning_phase(1)


Using TensorFlow backend.
/home/liy0f/.conda/envs/tf_newest_py2/lib/python2.7/site-packages/keras_applications/resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [2]:
# Freeze the layers except the last 4 layers
for layer in resnet.layers[:-4]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in resnet.layers:
#     print(layer, layer.trainable)
    if layer.name.startswith('bn'):
        layer.call(layer.input, training=False)

In [3]:
from keras import models
from keras import layers
from keras import optimizers
 
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(resnet)
 
# Add new layers

model.add(layers.Flatten())
# model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
# model.summary()

In [4]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [5]:
# train_data_dir = 'data/train'
# validation_data_dir = 'data/validation'
train_data_dir = 'chest_xray/train'
validation_data_dir = 'chest_xray/test'

In [6]:
epochs = 10
batch_size = 64

In [7]:
img_height, img_width = 224,224

In [8]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1. / 255)
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 5232 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [9]:
model.fit_generator(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator)

Epoch 1/10
82/82 [==============================] - 87s 1s/step - loss: 0.3458 - acc: 0.9287 - val_loss: 0.6679 - val_acc: 0.8574
Epoch 2/10
82/82 [==============================] - 85s 1s/step - loss: 0.0732 - acc: 0.9775 - val_loss: 0.8486 - val_acc: 0.8510
Epoch 3/10
82/82 [==============================] - 88s 1s/step - loss: 0.0347 - acc: 0.9886 - val_loss: 0.9541 - val_acc: 0.8510
Epoch 4/10
82/82 [==============================] - 88s 1s/step - loss: 0.0198 - acc: 0.9928 - val_loss: 0.8518 - val_acc: 0.8638
Epoch 5/10
82/82 [==============================] - 86s 1s/step - loss: 0.0202 - acc: 0.9931 - val_loss: 0.8435 - val_acc: 0.8638
Epoch 6/10
82/82 [==============================] - 89s 1s/step - loss: 0.0148 - acc: 0.9939 - val_loss: 1.0645 - val_acc: 0.8413
Epoch 7/10
82/82 [==============================] - 88s 1s/step - loss: 0.0102 - acc: 0.9960 - val_loss: 0.8923 - val_acc: 0.8526
Epoch 8/10
82/82 [==============================] - 86s 1s/step - loss: 0.0115 - acc: 0.99